In [1]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

In [ ]:
# pip install accelerate bitsandbytes
import torch
import requests
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

model_name="Salesforce/blip2-flan-t5-xl"
# model_name="Salesforce/blip2-opt-6.7b-coco"
processor = Blip2Processor.from_pretrained(model_name)
model = Blip2ForConditionalGeneration.from_pretrained(model_name, load_in_8bit=True, device_map="auto")


### Sandbox

In [11]:
# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# question = "give a caption describing the action of the dog"

# # Image captioning (without providing a text prompt):
# inputs = processor(raw_image, question, return_tensors="pt").to("cuda", torch.float16)

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))

playing with a woman on the beach


In [7]:
# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# # Image captioning (without providing a text prompt):
# inputs = processor(raw_image, return_tensors="pt").to("cuda", torch.float16)

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))

woman playing with dog on beach


In [5]:
# Half Precision
# processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl", torch_dtype=torch.float16, device_map="auto")

# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# question = "how many dogs are in the picture?"
# inputs = processor(raw_image, question, return_tensors="pt").to("cuda", torch.float16)

# Full Precision
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl", device_map="auto")
# question = "how many dogs are in the picture?"
# inputs = processor(raw_image, question, return_tensors="pt").to("cuda")


### Winoground

In [3]:
from dotenv import load_dotenv
import os
from tqdm import tqdm
import json

load_dotenv()  # This loads the environment variables from .env

api_key = os.getenv('HGF_KEY')

In [4]:
from datasets import load_dataset
winoground = load_dataset('facebook/winoground', token=api_key)['test']

In [5]:
SAMPLE_SIZE=5
NUM_BEAMS=5
DO_SAMPLE=True

In [6]:
example = winoground[7]
# winoground dataset images are already in RGB mode
raw_image_0 = example['image_0'] 
# # Image captioning (without providing a text prompt):
inputs = processor(raw_image_0, return_tensors="pt").to("cuda", torch.float16)

# question = ' '.join(example['caption_1'].split(' ')[:2])
# question = 'Question: is this the paired caption for this image: ' + example['caption_1']+ ' A:'
# print(question)
force_words = example['caption_0']
print(force_words)
force_words_ids = processor.tokenizer(force_words, add_special_tokens=False,return_tensors='pt').input_ids
print(force_words_ids.tolist())
# Image captioning prompted:
# inputs = processor(raw_image_0, question, return_tensors="pt").to("cuda", torch.float16)


# out = model.generate(**inputs,do_sample=True,num_beams=5,num_return_sequences=SAMPLE_SIZE,
# temperature=1)

out=model.generate(
    **inputs,
    force_words_ids=force_words_ids.tolist(),
    prefix_allowed_tokens_fn=lambda x,y:force_words_ids.tolist(),
    num_beams=5,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)
print(processor.batch_decode(out, skip_special_tokens=True))

a bottle is in water
[[102, 7304, 16, 11, 514]]


/mnt/efs/fs1/mmml/miniconda3/envs/cenv_x86/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['a ina bottle is inaaaa bottle isa,']


In [10]:
for o in out:
    print(processor.decode(o, skip_special_tokens=True))

a man in a cowboy hat kissing a little girl on the cheek

a man in cowboy hat kissing a little girl on the cheek

a man in cowboy hat kissing a little girl on the cheek

a man in a cowboy hat kissing a young girl on the cheek

a man in cowboy hat kissing a young girl on the cheek



In [7]:
def generate_on_data(winoground,f_stream):
    for example in tqdm(winoground):
        d = {}
        d['id'] = example['id']
    
        for i in range(2):     
            raw_image = example[f'image_{i}'] 
            inputs = processor(raw_image, return_tensors="pt").to("cuda", torch.float16)
            out = model.generate(**inputs,do_sample=DO_SAMPLE,
                                   num_beams=NUM_BEAMS,num_return_sequences=SAMPLE_SIZE)
            d[f'caption_{i}'] = [processor.decode(o, skip_special_tokens=True) for o in out]
    
        print(json.dumps(d),file=f_stream,flush=True)

In [10]:
with open(f'wino_{model_name.split("/")[-1]}_caps.jsonl','w') as f:
    generate_on_data(winoground,f)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [46:51<00:00,  7.03s/it]
